In [ ]:
from unet import UNet
import torch

unet_model_path = "unet_model.pth"

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
unet_model = UNet().to(device)
unet_model.load_state_dict(torch.load(unet_model_path))
unet_model.eval()

In [ ]:
from resnet import ResNetCNN

resnet_model_path = "resnet_model.pth"

#device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
resnet_model = ResNetCNN().to(device)
resnet_model.load_state_dict(torch.load(resnet_model_path))
resnet_model.eval()

In [3]:
from dataset import RESNETTestset
from torch.utils.data import DataLoader

test1_dir = '../chest_xray/test/NORMAL'
test2_dir = '../chest_xray/test/PNEUMONIA'

test1_dataset = RESNETTestset(unet_model, device, test1_dir, target_size=(256, 256))
test1_loader = DataLoader(test1_dataset, batch_size=1)

test2_dataset = RESNETTestset(unet_model, device, test2_dir, target_size=(256, 256))
test2_loader = DataLoader(test2_dataset, batch_size=1)

In [ ]:
with torch.no_grad():
    test1_len = 0
    test1_count = 0
    for idx, image in enumerate(test1_loader):
        image = image.to(device)
        output = resnet_model(image.repeat(1, 3, 1, 1))
        prediction = torch.sigmoid(output).cpu()
        eval = prediction[0, 0].item() > prediction[0, 1].item()
        #print(idx, prediction)
        
        test1_len += 1
        if eval:
            test1_count += 1
            
        #if idx > 10: break
            
    test2_len = 0
    test2_count = 0
    for idx, image in enumerate(test2_loader):
        image = image.to(device)
        output = resnet_model(image.repeat(1, 3, 1, 1))
        prediction = torch.sigmoid(output).cpu()
        eval = prediction[0, 0].item() < prediction[0, 1].item()
        #print(idx, prediction)
        
        test2_len += 1
        if eval:
            test2_count += 1
        
        #if idx > 10: break
    
    print(f'test1 : {100 * test1_count / test1_len:.2f}%')
    print(f'test2 : {100 * test2_count / test2_len:.2f}%')